In [140]:
# %% Importing libraries
from pathlib import Path
from re import M

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import csv
from plot_config import pblocks, NTERM_CLASSES, NTERM_COLORS, CTERM_CLASSES

## Global characterization of altered protein regions in the human annotation (GENCODE)

Dataset -> Gencode version 42 basic

In [142]:
# Reference-alternative protein isoform pairs in database
ref_alt_pair_unique = pblocks[['anchor','other']].drop_duplicates()
print(len((ref_alt_pair_unique)))

# Number of genes
# Value obtained from SQLite db = 20030

35083


### Altered protein regions across the proteome

In [ ]:
# Total altered protein regions
apr = len(pblocks)
print(apr)

# Average altered regions per isoform
avg = (apr/len(ref_alt_pair_unique))
print(avg)

# Isoform pairs contain a single altered region
print((pblocks.groupby(['anchor', 'other']).size() == 1).sum())

# Total number of isoform pairs
print(len((pblocks.groupby(['anchor', 'other']))))

# alternative isoforms contain two or more distinct regions
print((pblocks.groupby(['anchor', 'other']).size() > 1).sum())

# isoform pairs exhibited an extreme number of distinct altered regions
result = pblocks.groupby(['anchor', 'other']).size()
for name, group in result.items():
    if group > 10:
        print(name[0])

# The median number of affected amino acids

aa_loss = pblocks[pblocks['pblock_category'].isin({'DELETION', 'SUBSTITUTION'})].reset_index()[['pblock_category', 'aa_loss']]
aa_loss['pblock_category'].replace('SUBSTITUTION', 'SUBSTITUTION (reference)', inplace=True)
aa_loss.rename(columns={'aa_loss': 'length'}, inplace=True)
aa_gain = pblocks[pblocks['pblock_category'].isin({'INSERTION', 'SUBSTITUTION'})].reset_index()[['pblock_category', 'aa_gain']]
aa_gain['pblock_category'].replace('SUBSTITUTION', 'SUBSTITUTION (alternative)', inplace=True)
aa_gain.rename(columns={'aa_gain': 'length'}, inplace=True)
affected_lengths = pd.concat([aa_loss, aa_gain])
data = affected_lengths[affected_lengths['pblock_category'] != 'SUBSTITUTION (alternative)']
print(data.median())

# the first and third quartiles 
print(data.describe())

# Insertions, substituions and deletions altered protein region count
pblocks['pblock_category'].value_counts()

### N-terminal alterations

In [ ]:
def get_section(nterm, cterm):
    if nterm and cterm:
        return 'Full-length'
    elif nterm:
        return 'N-terminal'
    elif cterm:
        return 'C-terminal'
    else:
        return 'Internal'
pblocks['protein_section'] = list(map(get_section, ~pblocks['nterm'].isna(), ~pblocks['cterm'].isna()))
pblock_sections = pblocks['protein_section'].value_counts()
print(pblock_sections)

#### Transcriptional and post-transcriptional mechanisms by which different start ATGs are used

In [ ]:
# Mutually exclusive start, shared downstream start counts
print(pblocks['nterm'].value_counts())

from scipy.stats import mannwhitneyu
mxs_rel_lengths = nterm_pblocks[nterm_pblocks['nterm'] == 'Mutually exclusive starts']['anchor_relative_length_change'].abs()
sds_rel_lengths = nterm_pblocks[nterm_pblocks['nterm'] == 'Shared downstream start']['anchor_relative_length_change'].abs()

# Mann-Whitney U tests
mannwhitneyu(mxs_rel_lengths, sds_rel_lengths)

#### Translational and co-translational regulation that leads to differential N-terminal usage


In [ ]:
print(pblocks['nterm'].value_counts())


### Characterization of splicing patterns underlying internal protein region differences


#### Simple and compound splicing events

In [ ]:
internal_pblocks = (
    pblocks[pblocks['internal']].
    drop(columns=[col for col in pblocks.columns if 'start' in col or 'stop' in col]).
    copy()
)
# convert string repr back to Python object
internal_pblocks['tblock_events'] = internal_pblocks['tblock_events'].map(eval)
internal_pblocks['events'] = internal_pblocks['events'].map(eval)  

internal_subcats = pd.DataFrame(
    {
        'Frameshift': internal_pblocks['frameshift'],
        'Intron': internal_pblocks['tblock_events'].isin({('I',), ('i',)}),
        'Alt. donor': internal_pblocks['tblock_events'].isin({('D',), ('d',)}),
        'Alt. acceptor': internal_pblocks['tblock_events'].isin({('A',), ('a',)}),
        'Single exon': internal_pblocks['tblock_events'].isin({('E',), ('e',)}),
        'Compound': [True for _ in internal_pblocks.index]
    }
)
subcat_order = ('Intron', 'Alt. donor', 'Alt. acceptor', 'Single exon', 'Compound', 'Frameshift')
internal_pblocks['splice_event'] = internal_subcats.idxmax(axis=1).astype(pd.CategoricalDtype(subcat_order, ordered=True))


In [ ]:
internal_pblocks['splice_event'].value_counts()


In [ ]:
df = internal_pblocks[internal_pblocks['splice_event']== 'Single exon']

In [ ]:
internal_pblocks.query("pblock_category == 'DELETION'").sum()

In [ ]:
df.query("pblock_category == 'DELETION'").sum()

In [ ]:
internal_pblocks.sum()

### C-terminal alterations


In [ ]:
print(pblock_sections)

#### Characterization of direct splice-driven altered C-termini patterns

In [ ]:
cterm_pblocks = pblocks[~pblocks['cterm'].isna() & (pblocks['nterm'].isna()) & (pblocks['cterm'] != "ALTERNATIVE_ORF") & (pblocks['cterm'] != "UNKNOWN")].copy()
cterm_pblocks['cterm'] = cterm_pblocks['cterm'].map(CTERM_CLASSES).astype('category')
# Changed string to set for intersection
cterm_pblocks['APA'] = cterm_pblocks['events'].apply(lambda x: set(x).intersection('BbPp')).astype(bool)


In [ ]:
print(cterm_pblocks['cterm'].value_counts())

In [ ]:
cterm_pblock_events = cterm_pblocks['up_stop_events'].combine(cterm_pblocks['down_stop_events'], lambda x, y: (x, y))
single_ATE = (cterm_pblocks['cterm'] == 'Splice-driven') & cterm_pblocks['tblock_events'].isin({('B', 'b'), ('b', 'B')})
cterm_splice_subcats = pd.DataFrame(
    {
        'Exon extension introduces termination': cterm_pblocks['up_stop_events'].isin({'P', 'I', 'D'}),
        'Alternative terminal exon(s)': cterm_pblock_events.isin({('B', 'b'), ('b', 'B')}),
        'Poison exon inclusion': cterm_pblocks['up_stop_events'] == 'E',
        'Other': [True for _ in cterm_pblocks.index]
    }
)
cterm_pblocks['splice_subcat'] = cterm_splice_subcats.idxmax(axis=1).astype(pd.CategoricalDtype(cterm_splice_subcats.columns, ordered=True))


In [ ]:
cterm_pblocks['splice_subcat'].value_counts()

## Comparison of altered protein regions from GENCODE and predicted ORFs in a WTC11 experimental transcriptome

#### Protein sequence region differences across  WTC-11 PacBio data

Change input data in `plot_configure.py` to `../B_hybrid_aln_wtc11_v42/pblocks.tsv` and restart the kernel

In [1]:
from plot_config import pblocks, NTERM_CLASSES, NTERM_COLORS, CTERM_CLASSES

In [24]:
# Reference-alternative protein isoform pairs in database
ref_alt_pair_unique = pblocks[['anchor','other']].drop_duplicates()
print(len((ref_alt_pair_unique)))

# Number of genes
# Value obtained from SQLite db = 20202

44962


In [25]:
# Total altered protein regions
apr = len(pblocks)
print(apr)

# Average altered regions per isoform
avg = (apr/len(ref_alt_pair_unique))
print(avg)

# Isoform pairs contain a single altered region
print((pblocks.groupby(['anchor', 'other']).size() == 1).sum())

# Total number of isoform pairs
print(len((pblocks.groupby(['anchor', 'other']))))

# alternative isoforms contain two or more distinct regions
print((pblocks.groupby(['anchor', 'other']).size() > 1).sum())

# isoform pairs exhibited an extreme number of distinct altered regions
result = pblocks.groupby(['anchor', 'other']).size()
for name, group in result.items():
    if group > 10:
        print(name[0])

# The median number of affected amino acids

aa_loss = pblocks[pblocks['pblock_category'].isin({'DELETION', 'SUBSTITUTION'})].reset_index()[['pblock_category', 'aa_loss']]
aa_loss['pblock_category'].replace('SUBSTITUTION', 'SUBSTITUTION (reference)', inplace=True)
aa_loss.rename(columns={'aa_loss': 'length'}, inplace=True)
aa_gain = pblocks[pblocks['pblock_category'].isin({'INSERTION', 'SUBSTITUTION'})].reset_index()[['pblock_category', 'aa_gain']]
aa_gain['pblock_category'].replace('SUBSTITUTION', 'SUBSTITUTION (alternative)', inplace=True)
aa_gain.rename(columns={'aa_gain': 'length'}, inplace=True)
affected_lengths = pd.concat([aa_loss, aa_gain])
data = affected_lengths[affected_lengths['pblock_category'] != 'SUBSTITUTION (alternative)']
print(data.median())

# the first and third quartiles 
print(data.describe())

# Insertions, substituions and deletions altered protein region count
pblocks['pblock_category'].value_counts()

53915
1.1991237044615453
37307
44962
7655
length    121.0
dtype: float64
             length
count  53915.000000
mean     265.842511
std      407.249326
min        1.000000
25%       37.000000
50%      121.000000
75%      351.000000
max     7593.000000


/var/folders/yj/q7ccjqsx2rg2czn92rx0_8sc0000gn/T/ipykernel_83181/2711351022.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(data.median())


SUBSTITUTION    31532
DELETION        17716
INSERTION        4667
Name: pblock_category, dtype: int64

In [26]:
print(pblocks)

          anchor             other  pblock_number pblock_category  \
0       AAAS-201    AAAS|PB.9747.1              0        DELETION   
1       AAAS-201    AAAS|PB.9747.2              0        DELETION   
2       AAAS-201    AAAS|PB.9747.3              0        DELETION   
3       AAAS-201    AAAS|PB.9747.3              1    SUBSTITUTION   
4       AAAS-201    AAAS|PB.9747.4              0    SUBSTITUTION   
...          ...               ...            ...             ...   
53910    ZYX-201     ZYX|PB.6586.8              0    SUBSTITUTION   
53911    ZYX-201     ZYX|PB.6586.9              0    SUBSTITUTION   
53912  ZZEF1-201  ZZEF1|PB.12314.1              0       INSERTION   
53913   ZZZ3-202    ZZZ3|PB.677.12              0        DELETION   
53914   ZZZ3-202     ZZZ3|PB.677.2              0    SUBSTITUTION   

       pblock_anchor_start  pblock_anchor_stop  pblock_other_start  \
0                      148                 182                 148   
1                      489     